Есть корпус текстов. В нем есть выделенные при помощи каких-то мер устойчевые словосочетания. Надо организовать хранение и поиск текстов и словосочетаний. То есть, например, пользователь вводит сочетание, а мы ему показываем фрагменты текстов с ним (например, предложение), плюс метаинформацию по тексту.

In [10]:
import psycopg2

In [11]:
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from nltk import tokenize
import re
import string

# Создаём БД

In [12]:
con = psycopg2.connect(dbname='glmtkhyl',
                       user='glmtkhyl',
                       host='dumbo.db.elephantsql.com',
                       password='0xPKFGY35IHI2CtXhZmzN5vP9fHY3RCo')

In [13]:
cur = con.cursor()

In [318]:
con.rollback()

In [254]:
#cur.execute('DROP TABLE texts')
cur.execute('DROP TABLE phrases')

In [255]:
#таблица текстов; формат (t_id - номер текста; text - список предложений оригинального текста; 
#norm_text - список нормализированных предложений,author - автор текста; title - название, year - год выпуска);
#cur.execute("CREATE TABLE texts (t_id text, text text, norm_text text, author text, title text, year integer);")

#таблица словосочетаний; формат (p_id - номер фразы, phrase - сама фраза, meaning - значение, 
#t_ids - список списков "номер текста - номер предложения";)
cur.execute("CREATE TABLE phrases (p_id integer, phrase text, meaning text,t_ids text);") 

# Нужные частые функции

In [16]:
morph = MorphAnalyzer()
punct = punctuation+'«»―…“”*№–'

In [17]:
def normalize(text):
    if type(text) is str:
        words = [word.strip(punct) for word in text.lower().split()]
        words = [morph.parse(word)[0].normal_form for word in words if word]
        return ' '.join(words)
    else:
        for idx, sent in enumerate(text):
            words = [word.strip(punct) for word in sent.lower().split()]
            words = [morph.parse(word)[0].normal_form for word in words if word]
            text[idx] = ' '.join(words)
        return text

In [18]:
def update():   
    cur.execute("SELECT phrase FROM phrases")
    phrases = cur.fetchall()
    for phrase in phrases:
        nums = get_sents(list(phrase)[0])
        cur.execute("UPDATE phrases SET t_ids = %s WHERE phrase = %s", (nums, phrase))
    return

# Положим в БД данные для экспериментов

Данные сейчас введу вручную, мелкими глупыми функциями, чтобы не заморачиваться. Аналогичные этим будут использоватья ниже, но я их буду переписывать, чтобы было красивее.

### Положим пару текстов

In [242]:
with open ('test_texts.txt', encoding = 'UTF-8') as f:
    texts = f.read()

with open ('test_texts.txt', encoding = 'UTF-8') as f:
    norm_texts = f.read()

In [243]:
texts = texts.split('\n\n')
norm_texts = norm_texts.split('\n\n')

In [244]:
norm_text = []
for n in range(len(texts)):
    texts[n] = texts[n].replace('\ufeff', '').replace('…', '.')
    texts[n] = texts[n].split('\n')
    norm_texts[n] = norm_texts[n].replace('\ufeff', '').replace('…', '.')
    norm_texts[n] = norm_texts[n].split('\n')
    texts[n][0] = tokenize.sent_tokenize(texts[n][0])
    norm_text.append(normalize(tokenize.sent_tokenize(norm_texts[n][0])))

In [76]:
norm_text[1]

['вообще многий поздно',
 'я не ответить потому что это замечание не предполагать ответ',
 'собственно почти весь быть уже ясно',
 'не вешать нос прапорщик произнести я совершенно механически',
 'так военный хирург в приёмный покой раз за разом привычно говорить свой искалечить пациент что-нибудь вроде мы ещё с вы водка выпить лейтенант',
 'я не вешать живо возразить дыбовский',
 'просто надо же реалистично смотреть на вещий']

In [71]:
texts[1]

[['― Вообще ― многое поздно.',
  'Я не ответил, потому что это замечание не предполагало ответа.',
  'Собственно, почти всё было уже ясно.',
  '― Не вешайте нос, прапорщик, ― произнёс я совершенно механически.',
  'Так военный хирург в приёмном покое раз за разом привычно говорит своим искалеченным пациентам что-нибудь вроде: «Мы ещё с вами водки выпьем, лейтенант» .',
  '― Я не вешаю, ― живо возразил Дыбовский.',
  '― Просто надо же реалистично смотреть на вещи.'],
 'Сергей Ястребов',
 'Лунная соната',
 '2007']

In [245]:
def fill_texts(texts, norm_text):
    for i in range (len(texts)):
        cur.execute('INSERT INTO texts VALUES (%s, %s, %s, %s, %s, %s);', 
                    [str(i), texts[i][0], norm_text[i], texts[i][1], texts[i][2], str(texts[i][3])])
    con.commit()
    return

In [246]:
fill_texts(texts, norm_text)

### Добавим несколько фраз

In [256]:
with open ('text_phrases.txt', encoding = 'UTF-8') as f:
    phrases = f.read()

In [257]:
phrases= phrases.split('\n\n')

In [258]:
ph = []
mea = []
for p in phrases:
    ph.append(p.split('\n')[0])
    mea.append(p.split('\n')[1])

In [259]:
def get_where(phrase):
    cur.execute("SELECT * FROM texts")
    texts = cur.fetchall()
    ph = normalize(phrase)
    num_sent = []
    for t in range(len(texts)): #t - номер блока про текст
        for s in range(len(texts[t][2].split(','))): #s - порядковый номер предложения; texts[t][2] - нормализованный текст;
            if ph in texts[t][2].split(',')[s].strip('"}{'):
                a = []
                a.append(t)
                a.append(s)
                num_sent.append(a)
    return num_sent

In [260]:
def fill_phrases(phrases, meas):
    for i in range(len(phrases)):
        num = get_where(phrases[i])

        cur.execute('INSERT INTO phrases VALUES (%s, %s, %s, %s);',
                           [str(i), phrases[i], meas[0], num])
    con.commit()
    return

In [261]:
fill_phrases(ph, mea)

In [262]:
cur.execute("SELECT * FROM phrases")
cur.fetchall()

[(0, 'бить баклуши', 'лениться', '{{10,0},{11,3}}'),
 (1, 'кот наплакал', 'лениться', '{{8,3},{9,3}}'),
 (2, 'ахилесова пята', 'лениться', '{}'),
 (3, 'бабье лето', 'лениться', '{{6,4},{7,3}}'),
 (4, 'бальзам на душу', 'лениться', '{{4,3},{5,3}}'),
 (5, 'битый час', 'лениться', '{}')]

### Поиск по выражениям

In [99]:
def print_text(all_info, mea):
    i = 0
    print('Значение: ' + mea + '\n')
    for text, s in all_info:
        print(str(i) + '. ' + s)
        print('Название текста: ' + text[0][4])
        print('Автор текста: ' + text[0][3])
        print('Год создания текста: ' + str(text[0][5]) + '\n')
        print('~~~')
        i+=1
    return
    

In [100]:
def get_texts(t_ids):
    all_info = []
    for t_id, s in t_ids: #t_id - id текста, s - номер предложения;
        a = []
        cur.execute("SELECT * FROM texts WHERE t_id LIKE '%s'" % (t_id))
        text = cur.fetchall() #text - блок текста;
        sent = text[0][1].strip('{"}').split('","')[int(s)]
        a.append(text)
        a.append(sent)
        all_info.append(a)
    return all_info

In [101]:
def search_in_base(p):
    cur.execute("SELECT * FROM phrases WHERE phrase LIKE '%s'" % (p))
    phrase = cur.fetchall()
    if len(phrase) is not 0:
        p_id = phrase[0][0]
        mea = phrase[0][2]
        t_ids = [n.split(',') for n in phrase[0][3].strip('{}').split('},{')] #список в списке; [[текст, страница]]
        all_info = get_texts(t_ids) #список в списке; [[блок про текст, предложение]]
        print_text(all_info, mea)
    else:
        print('Такого выражения в базе нет.')
    con.commit()
    return 

In [103]:
search_in_base('второе дыхание')

Такого выражения в базе нет.


In [104]:
search_in_base('бить баклуши')

Значение: лениться

0. Сколько бы ни было ошибок в выборе профессии ― ничего нет и быть не может хуже 16-летнего и тем более 18-летнего недоросля, который понятия не имеет о своем пути в жизни и идет в вуз (обычно все равно в какой) с единственной целью ― продлить елико возможно свое безмятежное детство, когда на занятиях можно бить баклуши, а все жизненные проблемы решать путем периодических собеседований родителей с учителями.
Название текста: Прогноз на завтра
Автор текста: Игорь Бестужев-Лада
Год создания текста: 1985

~~~
1. Конечно, гораздо приятнее бить баклуши в этом госпитальном плавучем салоне с кисейными занавесками и раздушенными куклами-сестрами, чем работать в грязи, в крови и разбитых теплушках.
Название текста: Повесть о жизни
Автор текста: К. Г. Паустовский
Год создания текста: 1954

~~~


# Добавление новой фразы

In [105]:
def get_sents(phrase):
    cur.execute("SELECT * FROM texts")
    texts = cur.fetchall()
    ph = normalize(phrase)
    num_sent = []
    for t in range(len(texts)): #t - номер блока про текст
        for s in range(len(texts[t][2].split(','))): #s - порядковый номер предложения; texts[t][2] - нормализованный текст;
            if ph in texts[t][2].split(',')[s].strip('"}{'):
                a = []
                a.append(t)
                a.append(s)
                num_sent.append(a)
    return num_sent

In [106]:
def add_phrase(phrase, mea):
    cur.execute("SELECT * FROM phrases WHERE phrase LIKE '%s'" % (phrase))
    p = cur.fetchall()
    if len(p) is 0:
        nums = get_sents(phrase)
        cur.execute("SELECT p_id FROM phrases")
        p_id= list(cur.fetchall()[len(cur.fetchall())-1])[0]+1
        cur.execute('INSERT INTO phrases VALUES (%s, %s, %s, %s);',
                           [str(p_id), phrase, mea, nums])
    else:
        print('Такое выражение уже есть в базе.')
    con.commit()
    return

In [35]:
con.rollback()

In [107]:
my_phrase = 'бить баклуши'
mea = 'лениться'

In [108]:
add_phrase(my_phrase, mea)

Такое выражение уже есть в базе.


In [263]:
my_phrase = 'белая ворона'
mea = 'непохожий на других'

In [264]:
add_phrase(my_phrase, mea)

In [265]:
cur.execute("SELECT * FROM phrases")
cur.fetchall()

[(0, 'бить баклуши', 'лениться', '{{10,0},{11,3}}'),
 (1, 'кот наплакал', 'лениться', '{{8,3},{9,3}}'),
 (2, 'ахилесова пята', 'лениться', '{}'),
 (3, 'бабье лето', 'лениться', '{{6,4},{7,3}}'),
 (4, 'бальзам на душу', 'лениться', '{{4,3},{5,3}}'),
 (5, 'битый час', 'лениться', '{}'),
 (6, 'белая ворона', 'непохожий на других', '{{2,3},{3,3}}')]

# Добавление текста

С каждым новым текстом нужно проходить по всем выражениям. Так как база небольшая, легче просто заново заколнять графу t_ids с индексами предложений. В идеале, конечно, просто добавлять новую пару.

In [282]:
with open ('new_text.txt', encoding = 'UTF-8') as f:
    text = f.read()

auth = 'Андрей Сидорин'
title = 'Искатель невидимых превращений'
year = '2012'

In [267]:
def add_text(text, auth, title, year):
    text = text.replace('…', '.').strip('\ufeff\n')
    norm_text = normalize(tokenize.sent_tokenize(text))
    cur.execute("SELECT t_id FROM texts")
    i =  max([int(a[0]) for a in cur.fetchall()])+1
    cur.execute('INSERT INTO texts VALUES (%s, %s, %s, %s, %s, %s);', 
                [str(i), text, norm_text, auth, title, year])
    update()
    con.commit()
    return

In [283]:
add_text(text, auth, title, year)

In [284]:
cur.execute("SELECT * FROM texts WHERE author = '%s'" % auth)
cur.fetchall()

[('12',
  'Самое поразительное, что это происходило во времена, когда ещё не был открыт закон Ома, практически не существовало электроизмерительных приборов, а наличие тока в цепи проверяли искрой и даже. на вкус. Слабая математическая подготовка так и осталась ахиллесовой пятой великого учёного. Как физик он очень далеко продвинулся в изучении электромагнитных силовых линий и электромагнитного поля. Однако для научного объяснения сути сделанных им открытий требовалась хорошая математическая обработка данных, что оказалось не по силам Фарадею. Лишь позднее его идеи получили блестящее развитие в математической теории электромагнитного поля, которую вывел Джеймс Клерк Максвелл. ',
  '{"самый поразительный что это происходить в время когда ещё не быть открытый закон ома практически не существовать электроизмерительный прибор а наличие ток в цепь проверять искра и даже","на вкус","слабый математический подготовка так и остаться ахиллесов пять великое учёный","как физик он очень далеко прод

# Удаление фразы

In [146]:
def delete_phrase(phrase):
    cur.execute("DELETE FROM phrases WHERE phrase = '%s'" % phrase)
    con.commit()
    return

In [272]:
delete_phrase('ахилесова пята')

In [273]:
cur.execute("SELECT * FROM phrases")
cur.fetchall()

[(0, 'бить баклуши', 'лениться', '{{10,0},{11,3}}'),
 (1, 'кот наплакал', 'лениться', '{{8,3},{9,3}}'),
 (3, 'бабье лето', 'лениться', '{{6,4},{7,3}}'),
 (4, 'бальзам на душу', 'лениться', '{{4,3},{5,3}}'),
 (5, 'битый час', 'лениться', '{}'),
 (6, 'белая ворона', 'непохожий на других', '{{2,3},{3,3}}')]

# Удаление текста

Ситуация такая же, как и с добавлением. Нужно переписывать данные по встречаемости.
Для удобства сделала удаление по названию текста.

In [274]:
def delete_text(title):
    cur.execute("DELETE FROM texts WHERE title = '%s'" % title)
    update()
    con.commit()
    return    

In [285]:
delete_text('Искатель невидимых превращений')

In [286]:
cur.execute("SELECT * FROM texts WHERE title = '%s'" % 'Искатель невидимых превращений')
cur.fetchall()

[]

# Редактирование текста

In [41]:
def change_author(new_author, text_id):
    cur.execute("UPDATE texts SET author = %s WHERE t_id = %s", (new_author, text_id))
    con.commit()
    return

In [42]:
def change_title(new_title, text_id):
    cur.execute("UPDATE texts SET title = %s WHERE t_id = %s", (new_title, text_id))
    con.commit()
    return

In [160]:
def change_year(new_year, text_id):
    cur.execute("UPDATE texts SET year = %s WHERE t_id = %s", (new_year, text_id))
    con.commit()
    return

In [279]:
change_year('2011', '0')

In [280]:
cur.execute("SELECT * FROM texts WHERE t_id = '%s'" % '0')
cur.fetchall()

[('0',
  '{"― Бедные.","― Настя совсем расстроилась.","― Да уж, им не позавидуешь, ― согласился папа.","― Но не надо вешать нос!","Разве мы с вами нашего Барсика не на улице нашли?","Даже с бездомными кошками случаются настоящие чудеса.","Однако что же мы стоим на морозе?"}',
  '{бедный,"настя совсем расстроиться","да уж имя не позавидовать согласиться папа","но не надо вешать нос","разве мы с вы наш барсик не на улица найти","даже с бездомный кошка случаться настоящий чудо","однако что же мы стоять на мороз"}',
  'Екатерина Каретникова',
  'Зимняя сказка ',
  3012)]

In [163]:
cur.execute("SELECT * FROM phrases")
cur.fetchall()

[(0, 'бить баклуши', 'лениться', '{{10,0},{11,3}}'),
 (2, 'ахилесова пята', 'лениться', '{}'),
 (3, 'бабье лето', 'лениться', '{{6,4},{7,3}}'),
 (4, 'бальзам на душу', 'лениться', '{{4,3},{5,3}}'),
 (5, 'битый час', 'лениться', '{}'),
 (6, 'белая ворона', 'непохожий на других', '{{2,3},{3,3}}')]

# Дополнительные функции

Просмотр всех выражений

In [561]:
cur.execute("SELECT phrase FROM phrases")
cur.fetchall()[0]

('бить баклуши',)

In [296]:
def get_all_phrases():
    cur.execute("SELECT phrase FROM phrases")
    phrases = cur.fetchall()
    for i in range(len(phrases)):
        print(str(i+1) + '. ' + list(phrases[i])[0])
    return

In [297]:
 get_all_phrases()

1. бить баклуши
2. кот наплакал
3. бабье лето
4. бальзам на душу
5. битый час
6. белая ворона


Просмотр всех названий текстов

In [298]:
def get_all_titles():
    cur.execute("SELECT title FROM texts")
    titles = cur.fetchall()
    for i in range(len(titles)):
        print(str(i+1) + '. ' + list(titles[i])[0])
    return

In [299]:
get_all_titles()

1. Лунная соната
2. Музыкальное чувство памяти
3. Пути к универсалиям
4. Естественный отбор
5. Храм Согласия 
6. Каменный мост
7. Обезьяна приходит за своим черепом
8. Многоточие сборки
9. Прощеное воскресенье
10. Прогноз на завтра
11. Повесть о жизни
12. Зимняя сказка 


Просмотр всех авторов текстов

In [300]:
def get_all_authors():
    cur.execute("SELECT author FROM texts")
    authors = cur.fetchall()
    for i in range(len(authors)):
        print(str(i+1) + '. ' + list(authors[i])[0])
    return

In [301]:
get_all_authors()

1. Сергей Ястребов
2. Елена Костюкович
3. С. С. Неретина
4. Максим Милованов
5. Вацлав Михальский
6. Александр Терехов
7. Ю. О. Домбровский.
8. Ю. И. Андреева
9. Вацлав Михальский
10. Игорь Бестужев-Лада
11. К. Г. Паустовский
12. Екатерина Каретникова
